# Step 1: Prepare the documents

In [1]:
import json

with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [6]:
#docs_raw[0]

In [4]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)
# every key should now be on the same level
documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

# Step 2: Create Embeddings using Pretrained Models

In [8]:
# This is a new library compared to the previous modules. 
# Please perform "pip install sentence_transformers==2.7.0"
from sentence_transformers import SentenceTransformer

# if you get an error do the following:
# 1. Uninstall numpy 
# 2. Uninstall torch
# 3. pip install numpy==1.26.4
# 4. pip install torch
# run the above cell, it should work
model = SentenceTransformer("all-mpnet-base-v2")

/home/onwunalu/.pyenv/versions/llm-zoomcamp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
len(model.encode("This is a simple sentence"))

768

In [11]:
documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

In [12]:
#created the dense vector using the pre-trained model
operations = []
for doc in documents:
    # Transforming the title into an embedding using the model
    doc["text_vector"] = model.encode(doc["text"]).tolist()
    operations.append(doc)

In [28]:
#documents[1]['text']
#operations[0]

# Step 3: Setup ElasticSearch connection


In [16]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': '55c413f85ce3', 'cluster_name': 'docker-cluster', 'cluster_uuid': '8hDqk1dbQQudzsKwhdPguA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

# Step 4: Create Mappings and Index
Mapping is the process of defining how a document, and the fields it contains, are stored and indexed.

Each document is a collection of fields, which each have their own data type.

We can compare mapping to a database schema in how it describes the fields and properties that documents hold, the datatype of each field (e.g., string, integer, or date), and how those fields should be indexed and stored

In [17]:
# meta data when creating index
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

In [18]:
index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True) # delete index if it exists
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

# Step 5: Add documents into index


In [29]:
# add the the whole document into the index
# the operation is es_eclient.index(index=index_name, document=doc) loop for each document
for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

KeyboardInterrupt: 

# Step 6: Create end user query

In [21]:
search_term = "windows or mac?"
vector_search_term = model.encode(search_term)

In [22]:
query = {
    "field": "text_vector", # specify the field to be used for search
    "query_vector": vector_search_term, # supply the query vector
    "k": 5, # find the 5 nearest candiate
    "num_candidates": 10000,  # group of documents the search engine is going to look into
}

In [23]:
# pass the query information into the search function, source is the groups to retrieve
# the retrieved documents are ranked in descreasing order of scores
res = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "course"])
res["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'aEe9t5ABp1Jxq06XDBPq',
  '_score': 0.7147919,
  '_source': {'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully'}},
 {'_index': 'course-questions',
  '_id': 'e0e9t5ABp1Jxq06XTxYx',
  '_score': 0.6134734,
  '_source': {'question': 'WSL instructions',
   'course': 'mlops-zoomcamp',
   'section': 'Module 1: Introduction',
   'text': 'If you wish to use WSL on your windows machine, here are the setup instructions:\nCommand: Sudo apt install wget\nGet Anaconda download address here. wget <download address>\nTurn on Docker Desktop WFree Download | AnacondaSL2\nCommand: git clone <github repository address>\nVSCODE on WSL\nJupyter: pip3 install jupyter\nAdded by Gregory Morris (gwm1980@gmail.com)\nAll in all softwares at

# Step 7: Perform Keyword search with Semantic Search (Hybrid/Advanced Search)


In [34]:
# Note: I made a minor modification to the query shown in the notebook here
# (compare to the one shown in the video)
# Included "knn" in the search query (to perform a semantic search) along with the filter  
knn_query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000
}

In [35]:
response = es_client.search(
    index=index_name,
    query={
        "match": {
            #"section": "General course-related questions"
            "course": "data-engineering-zoomcamp",
                 },
    },
    knn=knn_query,
    size=5,
    explain=True # shows information how the scoring is created
)


In [36]:
response["hits"]["hits"]

[{'_shard': '[course-questions][0]',
  '_node': 'ywtVdjdSSmGv8ncPAmEuQQ',
  '_index': 'course-questions',
  '_id': 'aEe9t5ABp1Jxq06XDBPq',
  '_score': 1.2132494,
  '_source': {'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully',
   'section': 'General course-related questions',
   'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'text_vector': [-0.026965489611029625,
    -0.0006261687958613038,
    -0.01662948727607727,
    0.052851494401693344,
    0.0547652393579483,
    -0.03133989870548248,
    0.029942581430077553,
    -0.04808563366532326,
    0.04467552900314331,
    0.005839464720338583,
    0.016233086585998535,
    0.012001162394881248,
    -0.031222283840179443,
    0.016600511968135834,
    -0.048869043588638306,
    -0.06496305018663406,
    0.046434227377176285,
    -0.009297752752900124,
    -0.06425285339355469,
    -0.0137327089905738

# References
https://logz.io/blog/elasticsearch-mapping/#:~:text=Within%20a%20search%20engine%2C%20mapping,indexes%20and%20stores%20its%20fields

https://www.sbert.net/docs/sentence_transformer/pretrained_models.html

https://www.elastic.co/search-labs/tutorials

https://www.elastic.co/search-labs/blog/text-similarity-search-with-vectors-in-elasticsearch